In [2]:
import numpy as np
import tensorflow as tf


from time import time
from matplotlib import pyplot as plt
import os

In [11]:
mydict = {'air_conditioner':0, 'car_horn':1, 'children_playing':2, 'dog_bark':3,
       'drilling':4, 'engine_idling':5, 'gun_shot':6, 'jackhammer':7, 'siren':8,
       'street_music':9}
import librosa
audio_file_path="D:\\shravanne-tasks\\Datasets\\UrbanSound8K\\audio\\fold6\\24364-4-0-11.wav"
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)
mfccs = np.mean(librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40).T,axis=0)
x=[]
x.append(mfccs)
x=np.array(x)
x=np.reshape(x,(x.shape[0],10,4,1))


# PREDICT USING REGULAR KERAS TRAINED MODEL FILE (h5). 


In [12]:

keras_model_size = os.path.getsize("D:\\shravanne-tasks\\Urbansound 8k\\model.h5")/1048576  #Convert to MB
print("Keras Model size is: ", keras_model_size, "MB")
#Using regular keral model
model = tf.keras.models.load_model("D:\\shravanne-tasks\\Urbansound 8k\\model.h5")

time_before=time()
keras_prediction = model.predict(x)
time_after=time()
keras_prediction= np.round_(keras_prediction)
a,b=np.where(keras_prediction==1)

total_keras_time = time_after - time_before
print("Total prediction time for keras model is: ", total_keras_time)
for gerne, classs in mydict.items(): 
    if classs == b[0]:
        print("Total prediction time for keras model is: ",gerne)


Keras Model size is:  0.852630615234375 MB
1/1 [==============================] - 0s 163ms/step
Total prediction time for keras model is:  0.2540559768676758
Total prediction time for keras model is:  drilling


# PREDICT USING tflite 


In [13]:

tflite_size = os.path.getsize("D:\\shravanne-tasks\\Urbansound 8k\\model.tflite")/1048576  #Convert to MB
print("tflite Model without opt. size is: ", tflite_size, "MB")
#Not optimized (file size = 540MB). Taking about 0.5 seconds for inference
tflite_model_path = "D:\\shravanne-tasks\\Urbansound 8k\\model.tflite"

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()


tflite Model without opt. size is:  0.27533721923828125 MB


In [14]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [15]:
# Test the model on input data.
input_shape = input_details[0]['shape']
print(input_shape)


[ 1 10  4  1]


In [16]:
output_shape = output_details[0]['shape']
print(output_shape)

[ 1 10]


In [17]:
# Load image
input_data = x

interpreter.set_tensor(input_details[0]['index'], input_data)

time_before=time()
interpreter.invoke()
time_after=time()
total_tflite_time = time_after - time_before
print("Total prediction time for tflite without opt model is: ", total_tflite_time)

output_data_tflite = interpreter.get_tensor(output_details[0]['index'])
print("The tflite w/o opt prediction for this image is: ", output_data_tflite)
output_data_tflite=np.round_(output_data_tflite)
a,b=np.where(output_data_tflite==1)
for gerne, classs in mydict.items(): 
    if classs == b[0]:
        print("\nTotal prediction time for keras model is: ",gerne)


Total prediction time for tflite without opt model is:  0.0
The tflite w/o opt prediction for this image is:  [[2.6501674e-33 7.2916050e-17 1.9970786e-02 3.2749251e-06 9.8002601e-01
  8.1525699e-26 3.1213114e-20 1.3248617e-28 2.1428034e-23 2.4167195e-16]]

Total prediction time for keras model is:  drilling
